In [1]:
from keras.models import load_model
import cv2
import numpy as np
from keras.models import model_from_json
from pathlib import Path
from threading import Thread

In [2]:
f = Path("model/model_structure.json")
model_structure = f.read_text()
model = model_from_json(model_structure)
model.load_weights("model/model_weights.h5")

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

labels_dict={0:'MASK', 1:'MOUTH_BELOW', 2:'NO MASK', 3:'NOSE_BELOW'}
color_dict={0:(0,255,0),1:(0,0,255),2:(0,0,255),3:(0,0,255)}

In [5]:
source=cv2.VideoCapture(0)
n = 0
while(True):
    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)
    for x,y,w,h in faces:
        face_img=img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(64,64))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,64,64,3))
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)   
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1) #Expect an input from the user
           
    if (key==ord('q')):
        break
    if (key==ord('p')):
        cv2.setWindowTitle('LIVE', 'PAUSED')
        cv2.waitKey(-1)
        cv2.destroyAllWindows()
cv2.destroyAllWindows()
source.release()